In [ ]:
# For handling data
import os

# For text preprocessing
import re

# For NLP tasks
import nltk
from sklearn.model_selection import train_test_split

# Download NLTK data (only needed once)
nltk.download('punkt')
nltk.download('stopwords')

# Optional: for vectorization and building the model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression  # or any ML model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
# Load the two large text files (Only use this if you have two txt files)
with open("file_name.txt", "r", encoding="utf-8") as file1:
    text1 = file1.read()

with open("file_name2.txt", "r", encoding="utf-8") as file2:
    text2 = file2.read()

# Combine the content if needed
full_text = text1 + "\n" + text2

# Print basic info
print("Length of combined text:", len(full_text))
print("First 500 characters:\n", full_text[:500])

In [ ]:
# Step 2: Extract Q&A pairs using regex
qa_pairs = re.findall(r"Q[:：](.*?)A[:：](.*?)(?=Q[:：]|$)", full_text, re.DOTALL)

# Clean whitespace
qa_pairs = [(q.strip(), a.strip()) for q, a in qa_pairs if q.strip() and a.strip()]

# Preview
print(f"Extracted {len(qa_pairs)} Q&A pairs.")
for i in range(min(3, len(qa_pairs))):
    print(f"\nQ: {qa_pairs[i][0]}\nA: {qa_pairs[i][1]}")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the pairs into questions and answers
questions, answers = zip(*qa_pairs)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(questions)

import sklearn
print(sklearn.__version__)

# Split the pairs into questions and answers
questions, answers = zip(*qa_pairs)

#(Use this if you're using linear regression to build the chatbot)
from sklearn.linear_model import LogisticRegression

# Train the classifier on question-answer pairs
model = LogisticRegression(max_iter=1000)
model.fit(X, answers)

In [ ]:
# Function to clean speaker labels like "Parent:", "Doctor:"
def clean_answer(text):
    return re.sub(r'\b(Parent|Doctor|Patient):\s*', '', text).strip()

# Function to get the best matching answer
def get_best_answer(user_input, top_k=1, threshold=0.3):
    user_vec = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vec, X).flatten()

    top_indices = similarities.argsort()[::-1][:top_k]
    top_scores = similarities[top_indices]
    results = []

    for i, score in zip(top_indices, top_scores):
        if score >= threshold:
            return clean_answer(answers[i])

    #for i, score in zip(top_indices, top_scores):
     #   if score >= threshold:
      #      results.append((questions[i], answers[i]))

    if not results:
        return "I'm not sure about that. Could you please rephrase or ask a different question?"

    return clean_answer(results[0][1])

# Gradio interface function
def chatbot_interface(user_input):
    return get_best_answer(user_input)

# Launch Gradio app
gr.Interface(
    fn=chatbot_interface,
    inputs=gr.Textbox(lines=2, placeholder="Ask any question about..."),
    outputs="text",
    title="Chatbot Name",
    description="Ask questions from your knowledge base.",
    theme="default",
).launch(share=True)